In [775]:
import pandas as pd
import numpy as np
import time
from urllib import request
from bs4 import BeautifulSoup
import requests
import re
import os
import openai
import json
from openai.embeddings_utils import get_embedding, cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [885]:
api_key = "sk-"

In [ ]:
openai.api_key = api_key
openai.Model.list()

## Prompot function

In [691]:
def generate_prompt(text):    # define prompt for GTP3
        return """Choose the main event labeled with event name field. For this main event, put it in a json object in double quotes which consists of an event name, a time based on the event name, a date based on the event name, a description based on the event name, a location (where the event takes place at) based on the event name, an address based on the event name,an email based on the event name,a city based on the event name,a state based on the event name, a zip code integer based on the event name, a phone number based on the event name, . If there are HTML tags in the event, please ignore those tags. The JSON object should have the following keys: event name, date, time, description, location, email, address, city, state, zip, phone: \n{}:""".format(text)


In [949]:
def GPT_prompt(data, temperature,max_tokens, presence_penalty):
    urls = data[data.isna().any(axis=1)].url.tolist() # get urls for rows with missing values
    for url in urls:
        html = requests.get(url).text
        soups = BeautifulSoup(html, "html.parser")  # get html text from each urls
        
        text = ""                # get url from the link and get text only and append url text to it
        for soup in soups.find_all("p"): 
            text += "\n" + soup.get_text()
        for link in soups.find_all('a', 
                          attrs={'href': re.compile("^https://")}):
            text+= link.text
        if len(text) > 4000:        # if texts are greater than 4000 tokens, which is max request for the Davinci model, we only focus on main text (no header&footer)
            mains = soups.find('main')
            text = ""
            
            for main in mains.find_all("p"): 
                text += "\n" + main.get_text()
            for link in soups.find_all('a', 
                          attrs={'href': re.compile("^https://")}):
                text+= link.text
        
        response = openai.Completion.create(         # plug in text and call GTP3
        model="text-davinci-003",
        prompt=generate_prompt(text),
        temperature = temperature,
        max_tokens = max_tokens,
        presence_penalty = presence_penalty,
        
        best_of = 3)
        
        # load all json object
        json_object = json.loads(response["choices"][0]["text"])
        
        # assign correspondant fields in json object
        Time = json_object["time"]
        
        Location = json_object["location"]
        
        Address= json_object["address"]
        
        Content = json_object["description"]
        
        Date = json_object["date"]
        
        Phone = json_object["phone"]
        
        City = json_object["city"]
        
        State = json_object["state"]
        
        Zip = json_object["zip"]
        
        Email = json_object["email"]
        
    
        
        # fill the missing values
        print("check url = ", url)
        if data['Time'][data['url']==url].isna().tolist()[0]:
            print("missing url = ", url, " Time = ", Time)
            if Location != "":
                data['Time'][data['url']==url] = Time       
            else:
                data['Time'][data['url']==url] = "Not Applicable"   
                
        if data['Location'][data['url']==url].isna().tolist()[0]:
            if Location != "":
                data['Location'][data['url']==url] = Location
            else:
                data['Location'][data['url']==url] = "Not Applicable"
        
        if data['Address'][data['url']==url].isna().tolist()[0]:
            print(Address)
            if Address != "":
                data['Address'][data['url']==url] = Address
            else:
                data['Address'][data['url']==url] = "Not Applicable"
        
        if data['City'][data['url']==url].isna().tolist()[0]:
            data['City'][data['url']==url] = City
            if City != "":
                data['City'][data['url']==url] = City
            else:
                data['City'][data['url']==url] = "Not Applicable"
        
        if data['State'][data['url']==url].isna().tolist()[0]:
            data['State'][data['url']==url] = State
            if State != "":
                data['State'][data['url']==url] = State
            else:
                data['State'][data['url']==url] = "Not Applicable"            
        
        if data['Zip'][data['url']==url].isna().tolist()[0]:
            data['Zip'][data['url']==url] = Zip
            if Zip != "":
                data['Zip'][data['url']==url] = Zip
            else:
                data['Zip'][data['url']==url] = "Not Applicable"
        
        if data['Email'][data['url']==url].isna().tolist()[0]:
            data['Email'][data['url']==url] = Email
            if Email != "":
                data['Email'][data['url']==url] = Email
            else:
                data['Email'][data['url']==url] = "Not Applicable"
        
        if data['Phone'][data['url']==url].isna().tolist()[0]:
            data['Phone'][data['url']==url] = Phone
            if Phone != "":
                data['Phone'][data['url']==url] = Phone
            else:
                data['Phone'][data['url']==url] = "Not Applicable"
        
        if data['Time'][data['url']==url].isna().tolist()[0]:
            data['Time'][data['url']==url] = Time
            if Time != "":
                data['Time'][data['url']==url] = Time
            else:
                data['Time'][data['url']==url] = "Not Applicable"
        
        if data['Content'][data['url']==url].isna().tolist()[0]:
            data['Content'][data['url']==url] = Content
            if Content != "":
                data['Content'][data['url']==url] = Content
            else:
                data['Content'][data['url']==url] = "Not Applicable"
        
        if data['Date'][data['url']==url].isna().tolist()[0]:
            data['Date'][data['url']==url] = Date
            if Date != "":
                data['Date'][data['url']==url] = Date
            else:
                data['Date'][data['url']==url] = "Not Applicable"
    
    
    return data
            
            

## Similarity Function

In [961]:
def similarity(predicted_data, true_data):

    similarity_matrix = {}
    for column_name in predicted_data.columns:
        similarity_matrix[column_name] = []
        for i in range(len(predicted_data["Name"])):
            time.sleep(3)
            print("i = ", i)
            resp = openai.Embedding.create(
                 input = [str(predicted_data[column_name][i]),str(true_data[column_name][i])],
                 engine = "text-similarity-ada-001")
            print(resp)
            embedding_a = resp["data"][0]["embedding"]
            embedding_b = resp["data"][1]["embedding"]
            similarity = np.dot(embedding_a,embedding_b)
            similarity_matrix[column_name].append(similarity)
    similarity_matrix = pd.DataFrame(similarity_matrix)        
            
    return similarity_matrix

## Accuracy Function

In [966]:
def accuracy(similarity_matrix):
    correct_field = 0
    total_field = 0
    matrix = similarity_matrix.values
    
    for field in matrix:
        for item in field:
            if item >= 0.90:
                correct_field = correct_field + 1
            total_field+=1
    return correct_field / total_field * 100

## Visiting Tahoe

In [847]:
tahoe = pd.read_csv("for_gtp3_2.csv")

In [936]:
tahoe

,Name,Date,Time,Location,Image,Email,Address,City,State,Zip,Phone,Content,url
0,South Lake Tahoe - State of the City 2022,20-Oct-22,5:30 pm - 7:30 pm,Beach Retreat & Lodge,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"3411 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150.0,Not Applicable,Attend the City’s State of the City Address on...,https://visitlaketahoe.com/event/state-of-the-...
1,Witch's Night Out at the Hangar Taproom,20-Oct-22,6:00 pm - 10:00 pm,Hangar Taproom & Bottle Shop,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"2401 Lake Tahoe Blvd, South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150.0,Not Applicable,It’s time to grab your brooms and witch’s ward...,https://visitlaketahoe.com/event/witchs-night-...
2,Magic Fusion at The Loft Theatre,20-Oct-22,7:00 pm - 8:00 pm,The Loft Theatre & Lounge at the Heavenly Village,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,NaN,South Lake Tahoe,CA,NaN,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...
3,Tallac After Dark,20-Oct-22,7:00 pm - 8:30 pm,Tallac Historic Site,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1 Heritage Way, South Lake Tahoe, 96150",South Lake Tahoe,CA,96150.0,Not Applicable,Tallac Historic Site volunteers and staff invi...,https://visitlaketahoe.com/event/tallac-after-...
4,Live Music at McP's Taphouse Tahoe,20-Oct-22,NaN,McP’s Taphouse Grill,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"4125 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",NaN,CA,NaN,Not Applicable,McP’s Taphouse Tahoe has live entertainment ni...,https://visitlaketahoe.com/event/live-music-at...
5,Atmosphere - Party Over Here Tour at Bally's L...,20-Oct-22,8:00 pm - 10:00 pm,Bally’s Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"55 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449.0,Not Applicable,"Thursday, October 20, 2022, Atmosphere Party O...",https://visitlaketahoe.com/event/atmosphere-pa...
6,Bloom Bar & Lounge Nightlife,"October 20 - 21, 2022",NaN,NaN,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,NaN,South Lake Tahoe,CA,NaN,Not Applicable,NaN,https://visitlaketahoe.com/event/bloom-bar-lou...
7,Dueling Pianos at Harveys Lake Tahoe,"October 21 - 22, 2022","October 21 - 22, 2022",Harveys Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"18 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449.0,Not Applicable,Join us at the Mountain Bar located on Harveys...,https://visitlaketahoe.com/event/dueling-piano...
8,Magic Fusion at The Loft Theatre,21-Oct-22,7:00 pm - 8:00 pm,NaN,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1021 Heavenly Village Way, South Lake Tahoe, C...",South Lake Tahoe,CA,96150.0,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...


In [708]:
tahoe_prediction = GPT_prompt(tahoe,0.7,1000)


Attractions
The Loft Theatre & Lounge at the Heavenly Village
October 20, 2022
7:00 pm - 8:00 pm
The Loft in the Heavenly Village, South Lake Tahoe, CA, is pleased to announce that their show MAGIC FUSION is back! 
Shows are performing nightly at 7:00 & 9:00 PM, with an additional KIDS FREE matinee every Sunday at 4:30 PM. (Buy one adult ticket, get one kids ticket for free on Sundays for the 4:30 pm show).
Tickets for all shows are now on sale online at TheLoftTahoe.com or through all Ticketmaster outlets.   
Their restaurant Taste At The Loft is open.  Table reservations are available online at TheLoftTahoe.com. The Loft has scaled their theatre and dining room to allow for the CDC and State recommended guidelines and to assure our guests the utmost in safety, comfort, and confidence.  This includes social distancing, protective equipment, and health checks for all our entire team, as well as cleaning, sanitizing, and disinfecting protocols. Dining room guests are encouraged to make

/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Address'][data['url']==url] = Address
/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Zip'][data['url']==url] = Zip
/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a


10 Things to do in December at Lake Tahoe
A Burger and a Beer in South Lake Tahoe
Entertainment & Live Music in South Lake Tahoe
Tips & Tricks To Plan The Best Lake Tahoe Vacation Ever
Two Perfect Winter Days in South Lake Tahoe
35

                            Hike, bike & get outside                            

Roads
Webcams
Music & Dance
McP’s Taphouse Grill
October 20, 2022
8:00 pm - 11:00 pm
McP’s Taphouse Tahoe has live entertainment nightly featuring the most popular local talent. Open daily from 11:00 a.m. to 11:00 pm. Parking is available underground at the Chateau.
Did you know we have 40 beers on tap? Each beer is numbered and if that beer’s number falls on the current date and a weekday, it is the “beer of the day”!
20 big TVs and live music daily. The outside seating area is dog-friendly and everything is family-friendly until 11 p.m.Check out their Facebook page for more info
Start Planning
Plan Your Trip Today!
Make planning easy with this guide to all the must-see vist

/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Time'][data['url']==url] = Time
/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['City'][data['url']==url] = City
/var/folders/8g/mjzsydpx2yd2dxp7w_ht3xk00000gn/T/ipykernel_4835/572623172.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie


10 Things to do in December at Lake Tahoe
A Burger and a Beer in South Lake Tahoe
Entertainment & Live Music in South Lake Tahoe
Tips & Tricks To Plan The Best Lake Tahoe Vacation Ever
Two Perfect Winter Days in South Lake Tahoe
35

                            Hike, bike & get outside                            

Roads
Webcams
Music & Dance
Bloom Bar & Lounge Hard Rock Tahoe
October 20 - 21, 2022
Tahoe’s best nightlife experience couldn’t be anywhere else. WE OWN THE NIGHT. Thursdays, Fridays and Saturdays, 10:30 pm Doors. VIP Table and Bottle Service Available, Call Brian 530-318-9287 or Mike 530-683-7382. 
Lake Tahoe’s favorite nightspot is blooming again this summer. Bloom Bar and Lounge is reopening for the summer on Memorial Day Weekend. Guests can enjoy live DJs and bottle service every Friday and Saturday through October with a few additional dates throughout the summer. Reservations by VIP table bottle service by calling 530-318-9287.  Door cover may apply.
Bloom’s DJ Lineup
L

In [937]:
tahoe_prediction

,Name,Date,Time,Location,Image,Email,Address,City,State,Zip,Phone,Content,url
0,South Lake Tahoe - State of the City 2022,20-Oct-22,5:30 pm - 7:30 pm,Beach Retreat & Lodge,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"3411 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,Attend the City’s State of the City Address on...,https://visitlaketahoe.com/event/state-of-the-...
1,Witch's Night Out at the Hangar Taproom,20-Oct-22,6:00 pm - 10:00 pm,Hangar Taproom & Bottle Shop,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"2401 Lake Tahoe Blvd, South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,It’s time to grab your brooms and witch’s ward...,https://visitlaketahoe.com/event/witchs-night-...
2,Magic Fusion at The Loft Theatre,20-Oct-22,7:00 pm - 8:00 pm,The Loft Theatre & Lounge at the Heavenly Village,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1021 Heavenly Village Way, South Lake Tahoe, C...",South Lake Tahoe,CA,96150,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...
3,Tallac After Dark,20-Oct-22,7:00 pm - 8:30 pm,Tallac Historic Site,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1 Heritage Way, South Lake Tahoe, 96150",South Lake Tahoe,CA,96150,Not Applicable,Tallac Historic Site volunteers and staff invi...,https://visitlaketahoe.com/event/tallac-after-...
4,Live Music at McP's Taphouse Tahoe,20-Oct-22,8:00 pm - 11:00 pm,McP’s Taphouse Grill,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"4125 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,McP’s Taphouse Tahoe has live entertainment ni...,https://visitlaketahoe.com/event/live-music-at...
5,Atmosphere - Party Over Here Tour at Bally's L...,20-Oct-22,8:00 pm - 10:00 pm,Bally’s Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"55 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,"Thursday, October 20, 2022, Atmosphere Party O...",https://visitlaketahoe.com/event/atmosphere-pa...
6,Bloom Bar & Lounge Nightlife,"October 20 - 21, 2022",10:30 pm,Bloom Bar & Lounge Hard Rock Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"50 US-50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,Tahoe’s best nightlife experience couldn’t be ...,https://visitlaketahoe.com/event/bloom-bar-lou...
7,Dueling Pianos at Harveys Lake Tahoe,"October 21 - 22, 2022","October 21 - 22, 2022",Harveys Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"18 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,Join us at the Mountain Bar located on Harveys...,https://visitlaketahoe.com/event/dueling-piano...
8,Magic Fusion at The Loft Theatre,21-Oct-22,7:00 pm - 8:00 pm,The Loft Theatre & Lounge at the Heavenly Village,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1021 Heavenly Village Way, South Lake Tahoe, C...",South Lake Tahoe,CA,96150,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...


In [944]:
tahoe_truth

,Name,Date,Time,Location,Image,Email,Address,City,State,Zip,Phone,Content,url
0,South Lake Tahoe - State of the City 2022,20-Oct-22,5:30 pm - 7:30 pm,Beach Retreat & Lodge,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"3411 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,Attend the City’s State of the City Address on...,https://visitlaketahoe.com/event/state-of-the-...
1,Witch's Night Out at the Hangar Taproom,20-Oct-22,6:00 pm - 10:00 pm,Hangar Taproom & Bottle Shop,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"2401 Lake Tahoe Blvd, South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,It’s time to grab your brooms and witch’s ward...,https://visitlaketahoe.com/event/witchs-night-...
2,Magic Fusion at The Loft Theatre,20-Oct-22,7:00 pm - 8:00 pm,The Loft Theatre & Lounge at the Heavenly Village,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1021 Heavenly Village Way, South Lake Tahoe, C...",South Lake Tahoe,CA,96150,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...
3,Tallac After Dark,20-Oct-22,7:00 pm - 8:30 pm,Tallac Historic Site,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1 Heritage Way, South Lake Tahoe, 96150",South Lake Tahoe,CA,96150,Not Applicable,Tallac Historic Site volunteers and staff invi...,https://visitlaketahoe.com/event/tallac-after-...
4,Live Music at McP's Taphouse Tahoe,20-Oct-22,8:00 pm - 11:00 pm,McP’s Taphouse Grill,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"4125 Lake Tahoe Blvd., South Lake Tahoe, CA 96150",South Lake Tahoe,CA,96150,Not Applicable,McP’s Taphouse Tahoe has live entertainment ni...,https://visitlaketahoe.com/event/live-music-at...
5,Atmosphere - Party Over Here Tour at Bally's L...,20-Oct-22,8:00 pm - 10:00 pm,Bally’s Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"55 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,"Thursday, October 20, 2022, Atmosphere Party O...",https://visitlaketahoe.com/event/atmosphere-pa...
6,Bloom Bar & Lounge Nightlife,"October 20 - 21, 2022",10:30 PM,Bloom Bar & Lounge Hard Rock Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"50 US-50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,Tahoe’s best nightlife experience couldn’t be ...,https://visitlaketahoe.com/event/bloom-bar-lou...
7,Dueling Pianos at Harveys Lake Tahoe,"October 21 - 22, 2022","October 21 - 22, 2022",Harveys Lake Tahoe,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"18 Hwy 50, Stateline, 89449",South Lake Tahoe,CA,89449,Not Applicable,Join us at the Mountain Bar located on Harveys...,https://visitlaketahoe.com/event/dueling-piano...
8,Magic Fusion at The Loft Theatre,21-Oct-22,7:00 pm - 8:00 pm,The Loft Theatre & Lounge at the Heavenly Village,https://visitlaketahoe.com/event/state-of-the-...,Not Applicable,"1021 Heavenly Village Way, South Lake Tahoe, C...",South Lake Tahoe,CA,96150,Not Applicable,Magic Fusion at the Loft Theatre & Lounge. A p...,https://visitlaketahoe.com/event/magic-fusion-...


In [887]:
similarity(tahoe_prediction, tahoe_truth)

,Name,Date,Time,Location,Image,Email,Address,City,State,Zip,Phone,Content,url
0,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
1,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
2,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
3,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
4,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
5,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
6,1.0,1.0,0.99583,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.954741,1.0
7,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
8,1.0,1.0,1.00000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0


0.95 is a good threshold for text similarity. Also, the model is more generous on longer texts.

In [893]:
tahoe_prediction["Time"][6]

'10:30 pm'

In [894]:
tahoe_truth["Time"][6]

'10:30 PM'

In [759]:
tahoe_prediction["Content"][6]

'Tahoe’s best nightlife experience couldn’t be anywhere else. WE OWN THE NIGHT. Thursdays, Fridays and Saturdays, 10:30 pm Doors. VIP Table and Bottle Service Available, Call Brian 530-318-9287 or Mike 530-683-7382.'

In [760]:
tahoe_prediction["Content"][6]

'Tahoe’s best nightlife experience couldn’t be anywhere else. WE OWN THE NIGHT. Thursdays, Fridays and Saturdays, 10:30 pm Doors. VIP Table and Bottle Service Available, Call Brian 530-318-9287 or Mike 530-683-7382.'

## Deyoung Mesume

In [776]:
deyoung = pd.read_csv("deyoung_original.csv")

In [777]:
deyoung

,Name,Date,Time,Location,Address,Content,Email,url,Image,City,State,Zip,Phone
0,Museum Store Sunday,11/27/22,9:30 am – 5:15 pm,de Young and Legion of Honor Museum Stores,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,"Celebrating six years, Museum Store Sunday is ...",contact@famsf.org,https://www.famsf.org/events/museum-store-sunday,Not Applicable,San Francisco,CA,94118.0,Not Applicable
1,Rehearsed Reading of Helen by Euripides,12/3/22,NaN,Gunn Theater,"100 34th Ave, San Francisco, CA 94121",This program is directed and presented by Care...,contact@famsf.org,https://www.famsf.org/events/rehearsed-reading...,Not Applicable,NaN,CA,94121.0,Not Applicable
2,Envision: Community Portrait Day,12/3/22,11am - 4pm,De Young Fine Arts Museums of San Francisco,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,NaN,publicprograms@famsf.org,https://www.famsf.org/events/envision-communit...,Not Applicable,San Francisco,NaN,NaN,Not Applicable
3,Artisan Trunk Show: Kavita Silks,12/3/22,9:30am - 5:15pm,NaN,NaN,The de Young Museum Store is delighted to host...,NaN,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,San Francisco,CA,94121.0,Not Applicable
4,Artisan Trunk Show: Cute as a Button,12/3/22,9:30am - 5:15pm,Legion of Honor,"100 34th Ave, San Francisco, CA 94121",The Legion of Honor Museum Store is pleased to...,contact@famsf.org,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,NaN,CA,NaN,Not Applicable
5,Legacy Society Lecture + Champagne Toast,12/5/22,1-3pm,De Young Fine Arts Museums of San Francisco,NaN,NaN,specialevents@famsf.org,https://www.famsf.org/events/legacy-society-le...,Not Applicable,San Francisco,CA,94123.0,Not Applicable


In [778]:
deyoung_predicted = GPT_prompt(deyoung,0.1,1000,0,0)

In [779]:
deyoung_predicted

,Name,Date,Time,Location,Address,Content,Email,url,Image,City,State,Zip,Phone
0,Museum Store Sunday,11/27/22,9:30 am – 5:15 pm,de Young and Legion of Honor Museum Stores,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,"Celebrating six years, Museum Store Sunday is ...",contact@famsf.org,https://www.famsf.org/events/museum-store-sunday,Not Applicable,San Francisco,CA,94118.0,Not Applicable
1,Rehearsed Reading of Helen by Euripides,12/3/22,9:30 am – 5:15 pm,Gunn Theater,"100 34th Ave, San Francisco, CA 94121",This program is directed and presented by Care...,contact@famsf.org,https://www.famsf.org/events/rehearsed-reading...,Not Applicable,San Francisco,CA,94121.0,Not Applicable
2,Envision: Community Portrait Day,12/3/22,11am - 4pm,De Young Fine Arts Museums of San Francisco,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,Join us for Free Saturdays at the de Young and...,publicprograms@famsf.org,https://www.famsf.org/events/envision-communit...,Not Applicable,San Francisco,CA,94118,Not Applicable
3,Artisan Trunk Show: Kavita Silks,12/3/22,9:30am - 5:15pm,de Young Museum,"50 Hagiwara Tea Garden Drive, San Francisco, C...",The de Young Museum Store is delighted to host...,info@famsf.org,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,San Francisco,CA,94121.0,Not Applicable
4,Artisan Trunk Show: Cute as a Button,12/3/22,9:30am - 5:15pm,Legion of Honor,"100 34th Ave, San Francisco, CA 94121",The Legion of Honor Museum Store is pleased to...,contact@famsf.org,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,San Francisco,CA,94121,Not Applicable
5,Legacy Society Lecture + Champagne Toast,12/5/22,1-3pm,De Young Fine Arts Museums of San Francisco,"Golden Gate Park, 50 Hagiwara Tea Garden Drive...",Free admission to the de Young Museum on Satur...,specialevents@famsf.org,https://www.famsf.org/events/legacy-society-le...,Not Applicable,San Francisco,CA,94123.0,Not Applicable


In [923]:
deyoung_true = pd.read_csv("deyoung_true.csv")

In [924]:
deyoung_true

,Name,Date,Time,Location,Address,Content,Email,url,Image,City,State,Zip,Phone
0,Museum Store Sunday,11/27/22,9:30 am – 5:15 pm,de Young and Legion of Honor Museum Stores,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,"Celebrating six years, Museum Store Sunday is ...",contact@famsf.org,https://www.famsf.org/events/museum-store-sunday,Not Applicable,San Francisco,CA,94118,Not Applicable
1,Rehearsed Reading of Helen by Euripides,12/3/22,2-3pm,Gunn Theater,"100 34th Ave, San Francisco, CA 94121",This program is directed and presented by Care...,contact@famsf.org,https://www.famsf.org/events/rehearsed-reading...,Not Applicable,San Francisco,CA,94121,Not Applicable
2,Envision: Community Portrait Day,12/3/22,11am - 4pm,De Young Fine Arts Museums of San Francisco,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,Join us for Envision: Community Portrait Day a...,publicprograms@famsf.org,https://www.famsf.org/events/envision-communit...,Not Applicable,San Francisco,CA,94118,Not Applicable
3,Artisan Trunk Show: Kavita Silks,12/3/22,9:30am - 5:15pm,De Young Fine Arts Museums of San Francisco,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,The de Young Museum Store is delighted to host...,info@famsf.org,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,San Francisco,CA,94118,Not Applicable
4,Artisan Trunk Show: Cute as a Button,12/3/22,9:30am - 5:15pm,Legion of Honor,"100 34th Ave, San Francisco, CA 94121",The Legion of Honor Museum Store is pleased to...,contact@famsf.org,https://www.famsf.org/events/artisan-trunk-sho...,Not Applicable,San Francisco,CA,94121,Not Applicable
5,Legacy Society Lecture + Champagne Toast,12/5/22,1-3pm,De Young Fine Arts Museums of San Francisco,Golden Gate Park 50 Hagiwara Tea Garden Drive ...,Legacy Society members are invited to high tea...,specialevents@famsf.org,https://www.famsf.org/events/legacy-society-le...,Not Applicable,San Francisco,CA,94118,Not Applicable


In [925]:
deyoung_similarity_matrix = similarity(deyoung_predicted, deyoung_true)

In [967]:
deyoung_similarity_matrix

,Name,Date,Time,Location,Address,Content,Email,url,Image,City,State,Zip,Phone
0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,0.979915,1.0
1,1.0,1.0,0.899131,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,0.979901,1.0
2,1.0,1.0,1.000000,1.000000,0.998983,0.896459,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
3,1.0,1.0,1.000000,0.902071,0.976541,1.000000,1.0,1.0,1.0,1.0,1.0,0.961443,1.0
4,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
5,1.0,1.0,1.000000,1.000000,0.995063,0.743376,1.0,1.0,1.0,1.0,1.0,0.953176,1.0


In [968]:
accuracy(deyoung_similarity_matrix)

96.15384615384616

In [921]:
deyoung_predicted["Email"][3]

'info@famsf.org'

In [922]:
deyoung_true["Email"][3]

'contact@famsf.org'

In [913]:
deyoung_predicted["Email"][4]

'contact@famsf.org'

In [914]:
deyoung_true["Email"][4]

'info@famsf.org'

In [786]:
deyoung_predicted["Time"][1]

'9:30 am – 5:15 pm'

In [785]:
deyoung_true["Time"][1]

'2-3pm'

In [788]:
deyoung_predicted["Location"][3]

'de Young Museum'

In [789]:
deyoung_true["Location"][3]

'De Young Fine Arts Museums of San Francisco'

In [917]:
deyoung_predicted["Address"][3]

'50 Hagiwara Tea Garden Drive, San Francisco, CA 94118'

In [918]:
deyoung_true["Address"][3]

'Golden Gate Park 50 Hagiwara Tea Garden Drive San Francisco, CA 94121'

In [919]:
deyoung_predicted["Address"][5]

'Golden Gate Park, 50 Hagiwara Tea Garden Drive, San Francisco, CA 94118'

In [920]:
deyoung_true["Address"][5]

'Golden Gate Park 50 Hagiwara Tea Garden Drive San Francisco, CA 94123'

In [929]:
deyoung_predicted["Content"][5]

'Free admission to the de Young Museum on Saturdays.'

In [931]:
deyoung_true["Content"][5]

'Legacy Society members are invited to high tea and a champagne toast in the de Young Café with welcome remarks by Selena Shadle, director of legacy giving, and Thomas P. Campbell, director and CEO. Remarks will be followed with a lecture by Reneé Dreyfus, George and Judy Marcus Distinguished Curator in Charge of Ancient Art, on Ramses the Great and the Gold of the Pharaohs. After the lecture guests will be able to view the exhibition privately. Ticket info This event is open to Legacy Society members and special guests. Advance reservations are required and will be accepted on a first-come first-served basis. Legacy Society members and special guests can RSVP by Monday, November 21, at specialevents@famsf.org or 415.750.7648. To request accessibility accommodations, please reach out to specialevents@famsf.org by Wednesday, November 23. Interested in joining the Legacy Society? Learn more.'

In [799]:
deyoung_true["Zip"][3]

94121

In [798]:
deyoung_predicted["Zip"][3]

94121.0

## Fresno Fair

In [954]:
fresno = pd.read_csv("Fresno_original.csv")
fresno

,Name,Date,Time,Location,Address,Content,url,Image,City,State,Zip,Phone,Email
0,2023 Hmong Cultural New Year Celebration,"Dec 29, 2022 - Jan 01, 2025",NaN,Grounds,NaN,"DATE: DECEMBER 29, 2022 - JANUARY 1, 2023 TIME...",https://www.fresnofair.com/events/2022/hmong-c...,Not Applicable,Fresno,NaN,NaN,NaN,Not Applicable


In [955]:
fresno_predicted = GPT_prompt(fresno,0.2,1000,-2)

check url =  https://www.fresnofair.com/events/2022/hmong-cultural-new-celebration-2023
missing url =  https://www.fresnofair.com/events/2022/hmong-cultural-new-celebration-2023  Time =  



In [877]:
#fresno_predicted = fresno_predicted.fillna("Not Found")

In [956]:
fresno_predicted

,Name,Date,Time,Location,Address,Content,url,Image,City,State,Zip,Phone,Email
0,2023 Hmong Cultural New Year Celebration,"Dec 29, 2022 - Jan 01, 2025",Not Applicable,Grounds,Not Applicable,"DATE: DECEMBER 29, 2022 - JANUARY 1, 2023 TIME...",https://www.fresnofair.com/events/2022/hmong-c...,Not Applicable,Fresno,Not Applicable,Not Applicable,Not Applicable,Not Applicable


In [957]:
fresno_true = pd.read_csv("Fresno_true.csv")

In [958]:
fresno_true

,Name,Date,Time,Location,Address,Content,url,Image,City,State,Zip,Phone,Email
0,2023 Hmong Cultural New Year Celebration,"Dec 29, 2022 - Jan 01, 2026",TBD,Grounds,"1124 S. Chance Avenue Fresno, CA 93702","DATE: DECEMBER 29, 2022 - JANUARY 1, 2023 TIME...",https://www.fresnofair.com/events/2022/hmong-c...,Not Applicable,Fresno,CA,93702,(559) 650-3250,Not Applicable


In [ ]:
freno_similarity_matrix = similarity(fresno_predicted, fresno_true)

In [963]:
freno_similarity_matrix

,Name,Date,Time,Location,Address,Content,url,Image,City,State,Zip,Phone,Email
0,1.0,0.99051,0.812015,1.0,0.708807,1.0,1.0,1.0,1.0,0.748672,0.733479,0.776288,1.0


In [969]:
accuracy(freno_similarity_matrix)

61.53846153846154

In [1]:
# overall accuracy score
# 0.9 as a match
# no prediction & no ground truth
# else not match 

# matched counts / total counts